In [1]:
import builtins

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from itertools import combinations
from datetime import datetime

import time

In [2]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

In [3]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [4]:
wait = WebDriverWait(driver, 10)

In [5]:
driver.get("https://rustypot.com/coinflip")

In [56]:
def get_possible_bet_values(inventory_items_values, max_items=3):
    possible_bet_values = {}
    for r in range(1, max_items):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [57]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()
    
    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )
        
        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text[1:]))
        
        driver.get("https://rustypot.com/coinflip")

        print("inventory items gathered: ", inventory_items_values)
        
        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        get_inventory_items()

In [58]:
inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34]

# inventory_items = [i for i in range(1, 21, 3)]
possible_bet_values = get_possible_bet_values(inventory_items_values)
possible_bet_values

{13.34: [13.34],
 17.38: [17.38],
 20.86: [20.86],
 26.68: [13.34, 13.34],
 30.72: [17.38, 13.34],
 31.75: [31.75],
 34.2: [20.86, 13.34],
 38.239999999999995: [20.86, 17.38],
 45.09: [31.75, 13.34],
 49.129999999999995: [31.75, 17.38],
 52.61: [31.75, 20.86],
 62.31: [62.31],
 63.5: [31.75, 31.75],
 75.65: [62.31, 13.34],
 79.69: [62.31, 17.38],
 83.17: [62.31, 20.86],
 94.06: [62.31, 31.75]}

In [59]:
inventory_items_values = get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)
print(possible_bet_values)

[2025-03-18 23:58:40]  inventory items gathered:  [61.33, 31.75, 20.89, 7.04, 5.67, 5.51, 5.1, 4.91, 4.43, 4.33, 3.72, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39]
[2025-03-18 23:58:40]  {2.39: [2.39], 3.72: [3.72], 4.33: [4.33], 4.43: [4.43], 4.78: [2.39, 2.39], 4.91: [4.91], 5.1: [5.1], 5.51: [5.51], 5.67: [5.67], 6.11: [3.72, 2.39], 6.720000000000001: [4.33, 2.39], 6.82: [4.43, 2.39], 7.04: [7.04], 7.300000000000001: [4.91, 2.39], 7.49: [5.1, 2.39], 7.9: [5.51, 2.39], 8.05: [4.33, 3.72], 8.06: [5.67, 2.39], 8.15: [4.43, 3.72], 8.63: [4.91, 3.72], 8.76: [4.43, 4.33], 8.82: [5.1, 3.72], 9.23: [5.51, 3.72], 9.24: [4.91, 4.33], 9.34: [4.91, 4.43], 9.39: [5.67, 3.72], 9.43: [5.1, 4.33], 9.53: [5.1, 4.43], 9.84: [5.51, 4.33], 9.94: [5.51, 4.43], 10.0: [5.67, 4.33], 10.01: [5.1, 4.91], 10.1: [5.67, 4.43], 10.42: [5.51, 4.91], 10.58: [5.67, 4.91], 10.61: [5.51, 5.1], 10.76: [7.04, 3.72], 10.77: [5.67, 5.1], 11.18: [5.67, 5.51], 11.370000000000001: [7.04, 4.33], 11.46999

In [60]:
bets = []
counter = 0

inventory_items_values = get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)

while True:

    active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
    coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
    
    # print(f"{len(coinflip_divs)} active games")
    for coinflip_div in coinflip_row_divs:
        coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
        coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
        
        if "items" in coinflip_items.text:
            # print("more than 5 items...")
            continue
            
        img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
        
        coinflip_items_values = []
        for img in img_elements:
            title = img.get_attribute("data-original-title")
            if "|" in title:
                price_str = title.split("$")[-1]
                price = float(price_str)
                coinflip_items_values.append(price)
    
        possible_tax_value = coinflip_value * 2 * 0.05
        cheapest_item_value = coinflip_items_values[-1]
        
        if cheapest_item_value < possible_tax_value:
            # print(f"not profitable")
            continue
            
        # to join a cf, i need to deposit minimum value
        for bet_value in possible_bet_values:
            if  coinflip_value * 0.9 < bet_value < coinflip_value * 0.92 and coinflip_value not in bets and coinflip_value < 15.0:
                
                ### PRESS JOIN ON SELECTED BET ###
                join_button = coinflip_div.find_element(By.TAG_NAME, "button")
                if "Join" in join_button.text:
                    join_button.click()
                else:
                    continue
                ### PRESS JOIN ON SELECTED BET ###
                
                
                ### SELECT THE ITEMS ###
                inventory_items_div_locator = (By.CLASS_NAME, "depositInventory")
                inventory_items_div = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(inventory_items_div_locator)
                )
                
                try:
                    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
                    inventory_items = WebDriverWait(driver, 3).until(
                        EC.presence_of_all_elements_located(inventory_items_locator)
                    )
                    
                    inventory_items_values = []
                    for inventory_item in inventory_items:
                        item_value = float(inventory_item.text[1:])
                        if item_value in possible_bet_values[bet_value]:
                            inventory_item.click()
                            possible_bet_values[bet_value].remove(item_value)

                except TimeoutException:
                    print(f"Timed out waiting for inventory items")
                    driver.get("https://rustypot.com/coinflip")
                    
                deposit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
                )
                deposit_button.click()
                ### SELECT THE ITEMS ###
                
                ### DEPOSIT AND CLOSE STEAM WINDOW ###
                accept_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
                )
                accept_button.click()
                
                # Step 2: Wait for the new window/tab to open
                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                
                # Step 3: Switch to the new window
                original_window = driver.current_window_handle
                new_window = [w for w in driver.window_handles if w != original_window][0]
                driver.switch_to.window(new_window)
                
                
                try:
                    warning_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((
                            By.XPATH,
                            "//*[contains(., 'have reported Blown for attempting trade scams')]"
                        ))
                    )
                    ok_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((
                            By.XPATH,
                            "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                            ))
                    )
                    ok_button.click()
                except TimeoutException:
                    print(f"no report appeared")

                
                confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
                )
                confirm_button.click()
                time.sleep(1)
                
                
                confirm_gift_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
                )
                confirm_gift_button.click()
                time.sleep(1)
                
                
                accept_trade_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
                )
                accept_trade_btn.click()
                time.sleep(1)
                
                driver.close()
                driver.switch_to.window(original_window)
                
                ### DEPOSIT AND CLOSE STEAM WINDOW ###
                
                print(f"{coinflip_value=} -> "
                      f"{coinflip_items_values=} -> "
                      f"Bet items= {possible_bet_values[bet_value]}")
                
                bets.append(coinflip_value)
                
                view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
                view_button.click()
                
                
                # Wait until the <p> inside #fliper-coin disappears
                try:
                    p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
                    WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
                except:
                    # If <p> is already gone, just wait for the new state
                    pass
                
                WebDriverWait(driver, 120).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
                )

                
                ### wait to check if win or lost ###
                
                # Wait for the parent div to appear
                creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))
                
                # Wait for the child img.pick within that div
                color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))
                
                # Get the src
                src = color_img.get_attribute("src")
                
                # Extract the color from the filename
                if "Red" in src:
                    creator_color = "Red"
                elif "Black" in src:
                    creator_color = "Black"
                else:
                    creator_color = "Unknown"
                # extract flip color
                
                # 1. Get the 'flipper' element
                flipper = driver.find_element(By.CLASS_NAME, "flipper")
                
                # 2. Get the 'transform' style attribute
                transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"
                
                # 3. Extract the degree
                import re
                match = re.search(r'rotateY\((\d+)deg\)', transform_style)
                if match:
                    degrees = int(match.group(1))
                    # Normalize the rotation within 360°
                    final_angle = degrees % 360
                
                    # Every 180° flips the face, so even number of 180s = same side
                    flips = (degrees // 180) % 2
                
                    flip_face = "Red" if flips == 1 else "Black"
                    print("Coin landed on:", flip_face)
                else:
                    flip_face = "Couldn't determine coin face."
                    print(flip_face)

                # extract flip color
                
                if creator_color == flip_face:
                    print(f"flip was lost!")
                else:
                    print(f"flip was won!")
    
                time.sleep(10)
                
                ### wait to check if win or lost ###
                driver.get("https://rustypot.com/coinflip")
                inventory_items_values = get_inventory_items()
        
                possible_bet_values = get_possible_bet_values(inventory_items_values)
                
                break
        # print(f"{coinflip_value=} -> {possible_tax_value=:.2f} -> {cheapest_item_value=:.2f}")
    counter += 1
    # if counter % 100 == 0:
    #     print("active - ", counter)
    time.sleep(1)

[2025-03-18 23:58:48]  inventory items gathered:  [61.33, 31.75, 20.89, 7.04, 5.67, 5.51, 5.1, 4.91, 4.43, 4.33, 3.72, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39]
[2025-03-19 00:01:43]  no report appeared
[2025-03-19 00:01:46]  coinflip_value=5.45 -> coinflip_items_values=[1.3, 1.3, 1.17, 0.92, 0.76] -> Bet items= []
[2025-03-19 00:02:02]  Coin landed on: Black
[2025-03-19 00:02:02]  flip was won!
[2025-03-19 00:02:15]  inventory items gathered:  [61.33, 31.75, 20.89, 7.04, 5.67, 5.51, 5.1, 4.43, 4.33, 3.72, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39]
[2025-03-19 00:02:55]  no report appeared
[2025-03-19 00:02:59]  coinflip_value=5.65 -> coinflip_items_values=[4.11, 1.54] -> Bet items= []
[2025-03-19 00:03:13]  Coin landed on: Red
[2025-03-19 00:03:13]  flip was lost!
[2025-03-19 00:03:26]  inventory items gathered:  [61.33, 31.75, 20.89, 7.04, 5.67, 5.51, 4.91, 4.43, 4.33, 3.72, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.3

KeyboardInterrupt: 

In [48]:
driver.switch_to.window(original_window)

In [216]:
confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
                )
confirm_button.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00910E93+39155]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x007AE3B6+772070]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x005648EE+5182]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x0059998A+105610]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00599C7A+106362]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x005C9B92+302738]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x005B1EC4+205252]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00593091+78737]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x005C8530+297008]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x005B1B86+204422]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00592947+76871]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00591CF5+73717]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00592764+76388]
	sqlite3_dbdata_init [0x00A165BC+589708]
	Microsoft::Applications::Events:

In [38]:
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((By.CSS_SELECTOR, "#fliper-coin > p"))
)

### wait to check if win or lost ###

# wait to flip the coin
time.sleep(3)

# extract our color 
creator_div = driver.find_element(By.CLASS_NAME, "creator-imgs")
color_img = creator_div.find_element(By.XPATH, "//img[@class='pick']")
src = color_img.get_attribute("src")

# Extract the color from the filename
if "Red" in src:
    creator_color = "Red"
elif "Black" in src:
    creator_color = "Black"
else:
    creator_color = "Unknown"
# extract flip color

# 1. Get the 'flipper' element
flipper = driver.find_element(By.CLASS_NAME, "flipper")

# 2. Get the 'transform' style attribute
transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"

# 3. Extract the degree
import re
match = re.search(r'rotateY\((\d+)deg\)', transform_style)
if match:
    degrees = int(match.group(1))
    # Normalize the rotation within 360°
    final_angle = degrees % 360

    # Every 180° flips the face, so even number of 180s = same side
    flips = (degrees // 180) % 2

    flip_face = "Red" if flips == 1 else "Black"
    print("Coin landed on:", flip_face)
else:
    flip_face = "Couldn't determine coin face."
    print(flip_face)

# extract flip color

if creator_color == flip_face:
    print(f"flip was lost!")
else:
    print(f"flip was won!")

[2025-03-18 18:42:46]  Coin landed on: Black
[2025-03-18 18:42:46]  flip was won!
